In [85]:
import glob
import os
import pandas
import cv2
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [16, 9]

IMAGE_DIR = "./Cropped_Images/"
# constants - capitalized

#['044', '3', '09082019', 'P15', 'WCS365-1.5', 'R']
FIELD_NAMES = ['uniqID', 'expt', 'planting_date', 'plate_num', 'treatment', 'platehalf']

In [89]:
def process_img(file_name):
    im = cv2.imread(file_name)
    grey = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    #generate "foreground" -> leaves + roots
    grey_denoised = cv2.medianBlur(grey, 5)
    #generate "background" -> plate + condensation
    grey_blur = cv2.medianBlur(grey, 151)
    
    #view both plots
    fig, axs = plt.subplots(1,2)
    axs[0].imshow(grey_denoised)
    axs[1].imshow(grey_blur)
    plt.savefig("./QA_Images/"+fields[0]+".jpg", bbox_inches='tight')
    plt.close()

    grey_diff = cv2.subtract(grey_blur, grey_denoised)
    _, binary = cv2.threshold(grey_diff, 15, 255, cv2.THRESH_BINARY)
    prop_plant = np.count_nonzero(binary) / (binary.shape[0] *  binary.shape[1])
    
    return prop_plant

In [95]:
sample_img = "./Cropped_Images/001_2_08262019_P1_Buffer_L.jpg"

process_img(sample_img)

print(fields)
print(fields[0])

['066', '4', '09232019', 'P27', 'NFIX10-1.5', 'R']
066


In [91]:
pattern = os.path.join(IMAGE_DIR,"*.jpg")
#print(pattern)

field_list = []


for f in sorted(glob.glob(pattern)):
    img_basename = os.path.basename(f)
    img_name, ext = os.path.splitext(img_basename)

    fields = img_name.split("_")

    assert len(fields) == len(FIELD_NAMES)
    #assertion error if it's false

    z = dict(zip(FIELD_NAMES, fields))
    
    z['pixel_percent'] = process_img(f)

    field_list.append(z)
    

In [92]:
df = pandas.DataFrame(field_list)

df 
#df.to_csv("plantbiomass.csv")

,uniqID,expt,planting_date,plate_num,treatment,platehalf,pixel_percent
0,001,2,08262019,P1,Buffer,L,0.099595
1,002,2,08262019,P1,N2C3,R,0.053907
2,003,2,08262019,P2,NFACC05,R,0.072301
3,004,2,08262019,P2,NFACC05-5.1,L,0.070142
4,005,2,08262019,P3,NFACC05-1.1,R,0.066093
...,...,...,...,...,...,...,...
61,062,4,09232019,P24,NFACC56-1.5,R,0.068261
62,063,4,09232019,P25,NFIX10,L,0.167796
63,064,4,09232019,P25,NFIX10-5.1,R,0.114933
64,065,4,09232019,P26,NFIX10-1.1,L,0.097940
